In [113]:
import pandas as pd
import requests
import pamo.queries
from pamo.conecctions_shopify import ConnectionsShopify
from datetime import datetime
date_str = datetime.now().strftime('%d%m%y')

In [114]:
GET_ID_PRODUCT = """
      products(first: 1, query: "skus") {{
        edges {{
          node {{
            id
            tags
            variants(first: 1) {{
              edges {{
                node {{
                  price
                  {compareAtPrice}
                  sku
              }}
            }}
          }}
        }}
      }}
    }}
"""

In [115]:

# aqui se va a construir la consulta, la idea es que cuando se haga un cambio, inicialmente se va a traer el dato que ya esta en shopi para generar el informe
GET_ID_PRODUCT = """{{
    products(first: 1, query: "sku:{sku}") {{
        edges {{
          node {{
            id
          	{tags}
            {title}
            variants(first: 1) {{
              edges {{
                node {{
                  {price}
                  {compareAtPrice}
                  sku
                  {barcode}
                  {inventoryQuantity}
              }}
            }}
          }}
        }}
      }}
  }}
}}"""

# variables = {
#               "input": 
#               {
#                 "id": df_to_update.iloc[i]['id'],
#                 "title": df_to_update.iloc[i]['title'],
#                 "tags": [df_to_update.iloc[i]['TAGS']],
#                 "variants": 
#                   [{
#                     "price": df_to_update.iloc[i]['price'],
#                     "compareAtPrice": [df_to_update.iloc[i]['compareAtPrice']]
#                     "compareAtPrice": [df_to_update.iloc[i]['barcode']]
#                     "compareAtPrice": [df_to_update.iloc[i]['inventoryQuantity']]
#                   }]
#               }
#             }

#  aqui simplemente se deben cambiar los valores por variables que vienen del df
var_test = {
  "input": {
    "id": "gid://shopify/Product/8639730188565", 
    "tags": ["NUEVA_ETIQUETA_1"],
    "variants": [
      {
        "price": "2000",
        "compareAtPrice": "30000"
      }
    ]
  }
}

In [116]:
del var_test['input']['variants'][0]['compareAtPrice']

In [117]:
var_test['input']['variants']

[{'price': '2000'}]

In [118]:
GET_ID_PRODUCT.format(sku='',tags='',title='', price='', compareAtPrice='', barcode='',inventoryQuantity='')

'{\n    products(first: 1, query: "sku:") {\n        edges {\n          node {\n            id\n          \t\n            \n            variants(first: 1) {\n              edges {\n                node {\n                  \n                  \n                  sku\n                  \n                  \n              }\n            }\n          }\n        }\n      }\n  }\n}'

In [119]:

GET_ID_PRODUCT = """
      products(first: 1, query: "sku:{skus}") {
        edges {
          node {
            id
          {tags}
            variants(first: 1) {
              edges {
                node {
                  {price}
                  {compareAtPrice}
                  sku
              }
            }
          }
        }
      }
    }
"""

UPTADE_PRODUCT = """
  mutation UpdateProduct($input: ProductInput!) {
    productUpdate(input: $input) {
      product {
        id
        title
      }
      userErrors {
        field
        message
      }
    }
  }
"""

In [120]:
df = pd.read_excel('input\TAGS.xlsx', dtype=str)

In [121]:
df.head(3)

,Handle,Title,Published,Variant SKU,Image Alt Text,Status,MARCA,TAGS
0,desague-de-push-automatico-sin-rebose-plastico,DESAGUE DE PUSH AUTOMATICO SIN REBOSE PLASTICO,V,GRI-PUSH02,NaN,active,BARU,"GRINETTO,PLOMERIA,REPUESTO ,DESAGUE,PLASTICO,S..."
1,monocontrol-de-lavamanos-cromo-poste-bajo-mixta,MONOCONTROL DE LAVAMANOS CROMO POSTE BAJO MIXTA,V,GRI-643,NaN,active,BARU,"GRINETTO,GRIFERIA,LAVAMANOS ,MONOCONTROL,POSTE..."
2,bc-griferia-lavaplatos-monocontrol-con-extensi...,Bc Griferia Lavaplatos Monocontrol Con Extensi...,V,AG07-432,Bc Griferia Lavaplatos Monocontrol Con Extensi...,active,BOCCHERINI,"BOCCHERINI,GRIFERIA,LAVAPLATOS,EXTENSIBLE ,IND..."


In [122]:
# define los cambios de nombre en las columnas del df

price_var = 'price'
compare_var = 'compareAtPrice'
barcode_var = 'barcode'
Quantity_var = 'inventoryQuantity'
sku_var = 'Variant SKU'
tags_var = 'TAGS'
title_var = 'Title'
columns = {sku_var:'SKU', tags_var:'TAGS', title_var:'TITLE', price_var:'PRICE', compare_var:'COMPARE_AT_PRICE', barcode_var: 'BARCODE', Quantity_var:'INVENTORY_QUANTITY'}

try:
    del columns['']
except:
    pass

In [123]:
columns

{'Variant SKU': 'SKU',
 'TAGS': 'TAGS',
 'Title': 'TITLE',
 'price': 'PRICE',
 'compareAtPrice': 'COMPARE_AT_PRICE',
 'barcode': 'BARCODE',
 'inventoryQuantity': 'INVENTORY_QUANTITY'}

In [124]:
# se renombran las columnas
df.rename(columns={'SKU':'CODIGO', 'tag':'TAGS'}, inplace=True )

In [125]:
cant_skus = df.shape[0]
df.loc[df.duplicated(['CODIGO'], keep=False), 'ESTADO'] = "SKU DUPLICADO"
skus_to_search = df['CODIGO'].values

KeyError: Index(['CODIGO'], dtype='object')

In [ ]:
shopi = ConnectionsShopify()
# se obtienen los ids de shopiy
responses=[]
for i in skus_to_search:
    query = GET_ID_PRODUCT.format(skus = i)
    response = shopi.request_graphql(query)
    try:
         responses.append(response.json()['data']['products']['edges'][0])    
         print(response.json()['data']['products']['edges'][0])
    except Exception as error:
        print(error)

{'node': {'id': 'gid://shopify/Product/8120281399573', 'tags': ['Cocina', 'CON_IMAGEN', 'Flexible', 'Griferia Cocina', 'INVENTARIADO', 'MERCADOLIBRE', 'NUBEPOS', 'PAMO', 'PAMO_IMPORTACION', 'SODIMAC'], 'variants': {'edges': [{'node': {'price': '260000.00', 'compareAtPrice': '260000.00', 'sku': 'KM056'}}]}}}
{'node': {'id': 'gid://shopify/Product/8545025065237', 'tags': ['Barra Seguridad', 'Barras Seguridad', 'CON_IMAGEN', 'Institucional', 'MERCADOLIBRE', 'PAMO', 'SEBASTIAN', 'SODIMAC'], 'variants': {'edges': [{'node': {'price': '385000.00', 'compareAtPrice': None, 'sku': 'PAMO_304ABAT'}}]}}}
{'node': {'id': 'gid://shopify/Product/8382938611989', 'tags': ['Baño', 'Baños', 'CON_IMAGEN', 'Duchas', 'MELI', 'MERCADOLIBRE', 'PAMO', 'Regadera', 'SODIMAC'], 'variants': {'edges': [{'node': {'price': '45000.00', 'compareAtPrice': None, 'sku': 'PAMO_SQ031'}}]}}}
{'node': {'id': 'gid://shopify/Product/8382940414229', 'tags': ['SODIMAC'], 'variants': {'edges': [{'node': {'price': '65000.00', 'compa

In [ ]:
# se genera un df con los id de shopify y los datos a actualizar
data_shopi= { 'id': [item['node']['id'] for item in responses],
             'sku': [item['node']['variants']['edges'][0]['node']['sku'] for item in responses],
             'tags':[item['node']['tags'] for item in responses]} # esto se hace si queremos dejar las etiquetas que estan montadas y anexar otras
df_shopi = pd.DataFrame.from_dict(data_shopi)
df_update= df_shopi.merge(df, how='right', left_on='sku', right_on='CODIGO')
df_to_update = df_update.loc[df_update['sku'].notna()]



In [ ]:
df_to_update

,id,sku,tags,CODIGO,TAGS,ESTADO
0,gid://shopify/Product/8120281399573,KM056,"[Cocina, CON_IMAGEN, Flexible, Griferia Cocina...",KM056,SODIMAC,NaN
1,gid://shopify/Product/8545025065237,PAMO_304ABAT,"[Barra Seguridad, Barras Seguridad, CON_IMAGEN...",PAMO_304ABAT,SODIMAC,NaN
2,gid://shopify/Product/8382938611989,PAMO_SQ031,"[Baño, Baños, CON_IMAGEN, Duchas, MELI, MERCAD...",PAMO_SQ031,SODIMAC,NaN
3,gid://shopify/Product/8382940414229,PAMO_SQ032,"[SODIMAC, SODIMAC]",PAMO_SQ032,SODIMAC,NaN
4,gid://shopify/Product/8378132136213,PAMO_SR011,"[SODIMAC, SODIMAC]",PAMO_SR011,SODIMAC,NaN
...,...,...,...,...,...,...
308,gid://shopify/Product/8096189350165,AG-6800N,"[BOCCHERINI, CROMADA, GRIFERIA, LAVAMANOS, LAV...",AG-6800N,SODIMAC,NaN
309,gid://shopify/Product/8545558724885,234802,"[SODIMAC, SODIMAC]",234802,SODIMAC,NaN
310,gid://shopify/Product/8545584906517,98404,"[SODIMAC, SODIMAC]",98404,SODIMAC,NaN
311,gid://shopify/Product/8829802447125,H-R4008,"[MARLON, SODIMAC, SODIMAC, SODIMAC]",H-R4008,SODIMAC,SKU DUPLICADO


In [ ]:
# esto se hace si queremos dejar las etiquetas que estan montadas y anexar otras
df_to_update['tags'] = df_to_update['tags'].apply(lambda x:', '.join(map(str, x)))
df_to_update.loc[~df_to_update['tags'].apply(lambda x :'SODIMAC' in x), 'TAGS'] = df_to_update.loc[~df_to_update['tags'].apply(lambda x :'SODIMAC' in x), 'tags'] + ', '+ df_to_update.loc[~df_to_update['tags'].apply(lambda x :'SODIMAC' in x), 'TAGS']

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_10008\1736522402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_update['tags'] = df_to_update['tags'].apply(lambda x:', '.join(map(str, x)))


In [ ]:
# se cargan los datos desde el df
for i in range(df_to_update.shape[0]):
    variables = {
                "input": {
                    "id": df_to_update.iloc[i]['id'],
                    "tags": [df_to_update.iloc[i]['TAGS']],
                }
            }
    print(variables)
    try:
        response = shopi.request_graphql(UPTADE_PRODUCT, variables)
        print(response.json())
    except Exception as error:
        print(error)
    

{'input': {'id': 'gid://shopify/Product/8120281399573', 'tags': ['Cocina, CON_IMAGEN, Flexible, Griferia Cocina, INVENTARIADO, MERCADOLIBRE, NUBEPOS, PAMO, PAMO_IMPORTACION, SODIMAC']}}
{'data': {'productUpdate': {'product': {'id': 'gid://shopify/Product/8120281399573', 'title': 'Grifería Para Cocina Cuello Flexible Monocontrol Negra'}, 'userErrors': []}}, 'extensions': {'cost': {'requestedQueryCost': 10, 'actualQueryCost': 10, 'throttleStatus': {'maximumAvailable': 1000.0, 'currentlyAvailable': 990, 'restoreRate': 50.0}}}}
{'input': {'id': 'gid://shopify/Product/8545025065237', 'tags': ['Barra Seguridad, Barras Seguridad, CON_IMAGEN, Institucional, MERCADOLIBRE, PAMO, SEBASTIAN, SODIMAC']}}
{'data': {'productUpdate': {'product': {'id': 'gid://shopify/Product/8545025065237', 'title': 'Barra De Seguridad Abatible 80 Cms En Acero Inoxidable'}, 'userErrors': []}}, 'extensions': {'cost': {'requestedQueryCost': 10, 'actualQueryCost': 10, 'throttleStatus': {'maximumAvailable': 1000.0, 'curre

ConnectionError: HTTPSConnectionPool(host='prueba-1615.myshopify.com', port=443): Max retries exceeded with url: /admin/api/2021-07/graphql.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000194E0B02850>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# Revisiones finales
df_update.loc[df_update['sku'].isna(), "estado"] = 'con novedad'
df_update.loc[df_update["estado"].isna(), "estado"] = 'Actualizado correctamente'
print(df_update.groupby('estado')['CODIGO'].count())
df_update.to_excel(f'output/{date_str}_tags.xlsx', index=False)


estado
Actualizado correctamente    240
con novedad                   73
Name: CODIGO, dtype: int64


_______________________


In [ ]:
df_update.loc[df_update["estado"] == 'con novedad']

,id,sku,tags,CODIGO,TAGS,ESTADO,estado
0,NaN,NaN,NaN,KM056,SODIMAC,NaN,con novedad
1,NaN,NaN,NaN,PAMO_304ABAT,SODIMAC,NaN,con novedad
2,NaN,NaN,NaN,PAMO_SQ031,SODIMAC,NaN,con novedad
3,NaN,NaN,NaN,PAMO_SQ032,SODIMAC,NaN,con novedad
4,NaN,NaN,NaN,PAMO_SR011,SODIMAC,NaN,con novedad
...,...,...,...,...,...,...,...
296,NaN,NaN,NaN,DUC-03,SODIMAC,NaN,con novedad
297,NaN,NaN,NaN,YW011,SODIMAC,NaN,con novedad
298,NaN,NaN,NaN,TRAX-15X15,SODIMAC,NaN,con novedad
299,NaN,NaN,NaN,AG-6800N,SODIMAC,NaN,con novedad


No tocar


Esto se hizo por que tenia muchas novedades no deveria pasar

In [ ]:
df_update.loc[df_update['estado'] == 'con novedad']['CODIGO'].values

array(['KM056', 'PAMO_304ABAT', 'PAMO_SQ031', 'PAMO_SQ032', 'PAMO_SR011',
       'PAMO_F20215', 'BYJ-Z001', 'PAMO_P04', 'PAMO_FV003', 'KM001',
       'TAP-C03', 'PAMO_SQ033', 'BC001', 'CF012', 'PAMO_FS002',
       'PAMO_FS008', 'PAMO_SR010', 'PAMO_SQ031N', 'TC-8028',
       'PAMO_S74805', 'PAMO_SH16P', 'PAMO_F20213', 'PAMO_F10008H',
       'MG-6145-2', 'PAMO_PH05', 'BYJ-B002S', 'TC-22A-616', 'PAMO_86800B',
       'PAMO_S74501L', 'PAMO_S74501M', 'KM006', 'HS-5004', 'JY1017',
       'KTL1622', 'KYL1571', '7706829947733', '7528530106298', 'HS-3030',
       'PAMO_PHO4', 'D0041', 'D002', 'R0071', 'SD2018N', 'DJ03FX',
       'PLA-108S', 'HC9702', 'YW151', 'D005', 'HC9703', 'PLA-821A',
       'DT05FX', 'I001', '31-033-99', '31-026-99', '31-016-99',
       'JY-74100', 'JY-61800', '31200201', 'YW145-07', 'YW144-07',
       'YW143-07', 'YW146-07', 'DAM-L58', 'TO6020001', 'TO6030001',
       'TO6050001', 'AL6020001', 'SD-2018', 'PERCHERO-JY907C',
       'PAPELERAM-907C', 'TO6040001', 'YW147-07', 

In [ ]:
responses=[]
for i in df_update.loc[df_update['estado'] == 'con novedad']['CODIGO'].values:
    query = GET_ID_PRODUCT.format(skus = i)
    response = shopi.request_graphql(query)
    try:
         responses.append(response.json()['data']['products']['edges'][0])    
         print(response.json()['data']['products']['edges'][0])
    except Exception as error:
         print(error)
        

{'node': {'id': 'gid://shopify/Product/8544211861781', 'tags': ['0', 'CON_IMAGEN', 'MERCADOLIBRE', 'PAMO'], 'variants': {'edges': [{'node': {'price': '229300.00', 'compareAtPrice': None, 'sku': '8101'}}]}}}
{'node': {'id': 'gid://shopify/Product/8120352669973', 'tags': ['0', 'CON_IMAGEN', 'MELI', 'MERCADOLIBRE', 'NUBEPOS', 'PAMO', 'SEBASTIAN'], 'variants': {'edges': [{'node': {'price': '340500.00', 'compareAtPrice': '398000.00', 'sku': '8169'}}]}}}
{'node': {'id': 'gid://shopify/Product/7602899583142', 'tags': ['0', 'CON_IMAGEN', 'MERCADOLIBRE', 'NUBEPOS', 'PAMO'], 'variants': {'edges': [{'node': {'price': '188900.00', 'compareAtPrice': None, 'sku': '88001'}}]}}}
{'node': {'id': 'gid://shopify/Product/8055710384405', 'tags': ['Cocina', 'CON_IMAGEN', 'Griferia Cocina', 'MELI', 'NUBEPOS', 'Sencilla', 'SMART'], 'variants': {'edges': [{'node': {'price': '35700.00', 'compareAtPrice': '44625.00', 'sku': 'HS-5004-3A'}}]}}}
list index out of range
{'node': {'id': 'gid://shopify/Product/8545553

In [ ]:
data_shopi= { 'id': [item['node']['id'] for item in responses],
             'sku': [item['node']['variants']['edges'][0]['node']['sku'] for item in responses],
             'tags':[item['node']['tags'] for item in responses]}
df_shopi = pd.DataFrame.from_dict(data_shopi)
df_update= df_shopi.merge(df, how='right', left_on='sku', right_on='CODIGO')
df_to_update = df_update.loc[df_update['sku'].notna()]



In [ ]:
df_to_update

,id,sku,tags,CODIGO,TAGS,ESTADO
10,gid://shopify/Product/8544211861781,8101,"[0, CON_IMAGEN, MERCADOLIBRE, PAMO]",8101,SODIMAC,NaN
22,gid://shopify/Product/8120352669973,8169,"[0, CON_IMAGEN, MELI, MERCADOLIBRE, NUBEPOS, P...",8169,SODIMAC,NaN
32,gid://shopify/Product/7602899583142,88001,"[0, CON_IMAGEN, MERCADOLIBRE, NUBEPOS, PAMO]",88001,SODIMAC,NaN
39,gid://shopify/Product/8545553416469,16165188,"[MELI, SEBASTIAN]",16165188,SODIMAC,NaN
42,gid://shopify/Product/7602888114342,160600100,"[0, CON_IMAGEN, MERCADOLIBRE, NUBEPOS, SEBASTI...",160600100,SODIMAC,NaN
43,gid://shopify/Product/8545571569941,305068,"[MELI, SEBASTIAN]",305068,SODIMAC,NaN
50,gid://shopify/Product/7602852528294,31500401,"[0, CON_IMAGEN, MELI, MERCADOLIBRE, NUBEPOS, S...",31500401,SODIMAC,NaN
56,gid://shopify/Product/8107302322453,31600400,"[0, CON_IMAGEN, MELI, SEBASTIAN]",31600400,SODIMAC,NaN
65,gid://shopify/Product/8545553580309,16165339,[],16165339,SODIMAC,NaN
66,gid://shopify/Product/8545572946197,31200301,"[CON_IMAGEN, MELI, NATHALIE, SEBASTIAN, SODIMAC]",31200301,SODIMAC,NaN


In [ ]:
# se cargan los datos desde el df
for i in range(df_to_update.shape[0]):
    variables = {
                "input": {
                    "id": df_to_update.iloc[i]['id'],
                    "tags": [df_to_update.iloc[i]['TAGS']],
                }
            }
    print(variables)
    response = shopi.request_graphql(UPTADE_PRODUCT, variables)
    print(response.json())

{'input': {'id': 'gid://shopify/Product/8544211861781', 'tags': ['SODIMAC']}}
{'data': {'productUpdate': {'product': {'id': 'gid://shopify/Product/8544211861781', 'title': 'Grifería de Sensor Automático 110 Voltios Cromada'}, 'userErrors': []}}, 'extensions': {'cost': {'requestedQueryCost': 10, 'actualQueryCost': 10, 'throttleStatus': {'maximumAvailable': 1000.0, 'currentlyAvailable': 990, 'restoreRate': 50.0}}}}
{'input': {'id': 'gid://shopify/Product/8120352669973', 'tags': ['SODIMAC']}}
{'data': {'productUpdate': {'product': {'id': 'gid://shopify/Product/8120352669973', 'title': 'Grifo Lavamanos Cuello De Cisne Con Sensor (manos Libres).'}, 'userErrors': []}}, 'extensions': {'cost': {'requestedQueryCost': 10, 'actualQueryCost': 10, 'throttleStatus': {'maximumAvailable': 1000.0, 'currentlyAvailable': 990, 'restoreRate': 50.0}}}}
{'input': {'id': 'gid://shopify/Product/7602899583142', 'tags': ['SODIMAC']}}
{'data': {'productUpdate': {'product': {'id': 'gid://shopify/Product/7602899583

______________

In [129]:
data_form = {'csrfmiddlewaretoken': '6ebe0KjE07ogpU515GMICAfobH5Q5zISOvoLPN3SgWANaVWtGdua1fdgUEmuEmhN', 'Handle': 'N/A', 'Variant~SKU': 'SKU', 'TAGS': 'Tags', 'no~': 'N/A'}

In [131]:
data_form

{'Handle': 'N/A', 'Variant~SKU': 'SKU', 'TAGS': 'Tags', 'no~': 'N/A'}

In [144]:
data_form = {'csrfmiddlewaretoken': '6ebe0KjE07ogpU515GMICAfobH5Q5zISOvoLPN3SgWANaVWtGdua1fdgUEmuEmhN', 'Handle': 'N/A', 'Variant~SKU': 'SKU', 'TAGS': 'Tags', 'no~': 'N/A'}

In [145]:
del(data_form['csrfmiddlewaretoken'])
Change_colums={}
for key, value in data_form.items():
    if value != 'N/A':
        key = key.replace('~',' ')
        Change_colums[key] = value
df = pd.read_excel('C:/Users/USUARIO/Downloads/1710.xlsx')
df.rename(columns=Change_colums, inplace=True)

In [152]:
df[[i for i in df.columns if i in ['N/A','Tags','Titulo','Proveedor','Precio', 'Precio comparación','SKU','Codigo barras','Stock'] ]]

,SKU,Tags
0,GRI-PUSH02,"PLOMERIA,REPUESTO ,DESAGUE,PLASTICO,SIN REBOSE,"
1,GRI-643,"GRIFERIA,LAVAMANOS ,MONOCONTROL,POSTE BAJO,,"
2,GRI-PUSH03,"PLOMERIA,REPUESTO ,BOTON,,,"
3,ZW-R-50,"PLOMERIA,REPUESTO,,,,"
4,YW148-07,"ACCESORIO,NEGRO,,,,"
...,...,...
181,PAMO_F1120,"GRIFERIA,ORINAL,PUSH,CROMADO,INSTITUCIONAL,"
182,"PAMO_6615P,PAMO_6615A,I001,I003,,ACOP_MET_002,...","GRIFERIA,INSTITUCIONAL,LAVAMANOS,LAVAPLATOS,VA..."
183,"PAMO_6615P,PAMO_6615A,I001,I003,LCAP36,ACOP_ME...","GRIFERIA,INSTITUCIONAL,LAVAMANOS,LAVAPLATOS,VA..."
184,PAMO_6630,"GRIFERIA,INSTITUCIONAL,LAVAMANOS,LAVAPLATOS,VA..."


In [140]:
rename_columns({'csrfmiddlewaretoken': '6ebe0KjE07ogpU515GMICAfobH5Q5zISOvoLPN3SgWANaVWtGdua1fdgUEmuEmhN', 'Handle': 'N/A', 'Variant~SKU': 'SKU', 'TAGS': 'Tags', 'no~': 'N/A'})

NameError: name 'rename_columns' is not defined

Index(['Handle', 'SKU', 'Tags', 'no '], dtype='object')